## 배낭 (Knapsack) 문제

- 가방(Knapsack)에 물건을 여러 개 담을 때 가방에 담은 물건들의 조합의 가치를 극대화 할 수 있는 방법을 찾는 문제
- 다만, 배낭의 허용 용량을 초과하여 아이템을 담을 수 없다는 제약이 있음

In [2]:
import random
import math

class Knapsack:
    def __init__(self, items, capacity):
        self.items = items
        self.capacity = capacity

    def calculate_value(self, solution): # 총 가치 계산
        total_value = 0
        for i in range(len(self.items)):
            if solution[i] == 1:
                _, value, _ = self.items[i]
                total_value += value
        return total_value

    def measure_weight(self, solution): # 무게 측정
        total_weight = 0
        for i in range(len(self.items)):
            if solution[i] == 1:
                _, _, weight = self.items[i]
                total_weight += weight
        return total_weight

    def get_neighbors(self, solution):
        all_neighbors = []
        valid_neighbors = []
        for i in range(len(self.items)):
            neighbor = solution[:]
            neighbor[i] = 1 - neighbor[i]  # 물건 포함 여부를 뒤집음 (0 혹은 1)
            all_neighbors.append(neighbor)
        valid_neighbors = [n for n in all_neighbors if self.is_valid(n)]  # 무게 초과하지 않는 이웃 상태만 도출
        return valid_neighbors

    def is_valid(self, solution): 
        return self.measure_weight(solution) <= self.capacity # 아이템 총합 배낭 허용 용량 초과 여부 확인

    def random_initial_solution(self): # 초기 해 랜덤 생성
        while True:
            initial_solution = [random.randint(0, 1) for _ in range(len(self.items))]
            if self.is_valid(initial_solution):
                return initial_solution

In [3]:
# (이름, 가치, 무게) 튜플로 이루어진 아이템 목록
items = [
    ("책", 30, 1),
    ("노트북", 300, 3),
    ("물병", 20, 1),
    ("필통", 10, 1),
    ("간식", 50, 1),
    ("이어폰", 60, 0.5),
    ("공책", 15, 0.5),
    ("전공책", 100, 2)
]

# 배낭 용량
capacity = 5 

## 언덕 오르기 알고리즘

```java
Function HILL_CLIMBING(problem)
  current_solution ← problem.initial_solution
  while True
    next_solution ← current_solution의 이웃 상태 중 가장 높은 값을 가진 상태
    if next_solution의 값 > current_solution의 값
      current_solution ← next_solution
  return current_solution
```

In [7]:
import random

def hill_climbing(knapsack):
    # 초기 해 설정
    current_solution = knapsack.random_initial_solution()
    current_value = knapsack.calculate_value(current_solution)
    # print(f"초기값: {current_value}")
    # count = 1

    while True:
        # 현재 해의 이웃 상태들 계산
        neighbors = knapsack.get_neighbors(current_solution)
        # print(f"{count}번째 이웃상태: {[knapsack.calculate_value(n) for n in neighbors]}")

        # 이웃 중에서 가장 좋은 해 선택
        next_solution = None
        for neighbor in neighbors:
            neighbor_value = knapsack.calculate_value(neighbor)
            if neighbor_value > current_value:
                current_value = neighbor_value
                next_solution = neighbor

        # 반복을 다 돌고 난 이후에도 더 나은 해가 없으면 종료
        if next_solution is None:
            break
        
        # 더 나은 해로 이동
        current_solution = next_solution
        # count += 1

    # 최종 해와 그 가치를 반환
    return current_solution, current_value

In [4]:
initial_state = Knapsack(items, capacity)
solution, total_value = hill_climbing(initial_state)

# 결과 출력
selected_items = [items[i][0] for i in range(len(items)) if solution[i] == 1]
print(f"결과값: {total_value}, 아이템 목록: {solution}(={selected_items})")

초기값: 220
1번째 이웃상태: [210, 170, 160, 235, 120]
2번째 이웃상태: [225, 185, 175, 220, 135]
결과값: 235, 아이템 목록: [0, 0, 0, 1, 1, 1, 1, 1](=['필통', '간식', '이어폰', '공책', '전공책'])


## 무작위 재시작 언덕 오르기
- 지역 최댓값을 피하기 위해 무작위로 재시작하는 언덕오르기 반복해 전역 최댓값 도출

In [34]:
solutions = []
for i in range(100):
    initial_state = Knapsack(items, capacity)
    solution, total_value = hill_climbing(initial_state)
    print(f'{i+1}번: {solution}, {total_value}')
    solutions.append((solution, total_value))

final_solution = max(results, key = lambda x:x[1])
final_items = [items[i][0] for i in range(len(items)) if final_solution[0][i] == 1]
print(f'최종목록: {final_items}, 최종값: {final_solution[1]}')

1번: [0, 1, 1, 0, 1, 0, 0, 0], 370
2번: [0, 1, 0, 0, 1, 1, 1, 0], 425
3번: [1, 1, 0, 0, 0, 1, 1, 0], 405
4번: [1, 0, 1, 1, 0, 0, 0, 1], 160
5번: [1, 1, 0, 0, 0, 1, 1, 0], 405
6번: [0, 1, 0, 0, 0, 0, 0, 1], 400
7번: [0, 0, 0, 1, 1, 1, 1, 1], 235
8번: [0, 1, 1, 0, 1, 0, 0, 0], 370
9번: [0, 1, 1, 0, 0, 1, 1, 0], 395
10번: [1, 0, 0, 1, 1, 0, 0, 1], 190
11번: [1, 0, 1, 0, 0, 1, 1, 1], 225
12번: [1, 0, 1, 0, 0, 1, 1, 1], 225
13번: [0, 0, 0, 1, 1, 1, 1, 1], 235
14번: [0, 1, 0, 0, 0, 0, 0, 1], 400
15번: [0, 1, 0, 0, 1, 1, 1, 0], 425
16번: [0, 1, 0, 1, 0, 1, 1, 0], 385
17번: [0, 1, 0, 0, 1, 1, 1, 0], 425
18번: [0, 1, 0, 0, 1, 1, 1, 0], 425
19번: [1, 1, 0, 0, 0, 1, 1, 0], 405
20번: [1, 0, 1, 1, 1, 1, 1, 0], 185
21번: [0, 0, 1, 0, 1, 1, 1, 1], 245
22번: [1, 0, 0, 0, 1, 1, 1, 1], 255
23번: [1, 0, 0, 1, 1, 0, 0, 1], 190
24번: [1, 0, 0, 0, 1, 1, 1, 1], 255
25번: [1, 0, 0, 0, 1, 1, 1, 1], 255
26번: [0, 1, 0, 1, 0, 1, 1, 0], 385
27번: [1, 1, 0, 0, 0, 1, 1, 0], 405
28번: [1, 0, 0, 1, 0, 1, 1, 1], 215
29번: [0, 1, 1, 0, 0, 1, 1, 0]

## 모의 정련(simulated annealing)


```java
Function SIMULATED_ANNEALING(problem)
  current_solution ← problem.initial_solution
  t ← 0
  while True
    T ← schedule(t)
    if T = 0
      return current_solution
    next_state ← current_solution의 이웃 상태 중 무작위 선택
    ΔE ← next_solution의 값 - current_solution의 값
    if ΔE > 0
      current_solution ← next_solution
    else
      current_solution ← next_solution #단, e^(ΔE/T)의 확률로!
    t += 1
  return current_solution
```

- 단, schedule의 경우 냉각 스케쥴링(t가 늘어남에 따라 온도를 어떻게 감소시킬지 결정) 함수 추가 필요 (본 수업에서는 cooling_rate을 도입하여, 시간이 지남에 따라 온도가 선형적으로 감소하도록 만들 계획)

In [35]:
def cooling_schedule(t, initial_temp, cooling_rate): 
    temp = initial_temp - cooling_rate * t #선형 감소 
    if temp < 0:
        temp = 0
    return temp

# Simulated Annealing 알고리즘 구현 (출력 포함)
def simulated_annealing(problem, initial_temp=1000, cooling_rate=1.0):
    current_solution = problem.random_initial_solution()
    current_value = problem.calculate_value(current_solution)
    print(f"초기값: {current_value}")
    
    t = 0
    while True:
        # 선형 감소 방식으로 온도 계산
        temp = cooling_schedule(t, initial_temp, cooling_rate)
        
        # 온도가 0이면 종료, 온도는 0 이하로 내려가면안됨
        if temp == 0:
            break
        
        neighbors = problem.get_neighbors(current_solution)
        # print(f"{t+1}번째 이웃상태: {[problem.calculate_value(n) for n in neighbors]}, 현재값: {current_value}")
        
        next_solution = random.choice(neighbors) # 현재 해의 이웃 중 무작위 선택
        next_value = problem.calculate_value(next_solution)

        # ΔE 계산
        delta_E = next_value - current_value

        if delta_E > 0: # 더 좋은 해로 이동
            current_solution = next_solution
            current_value = next_value
        else:
            if random.random() < math.exp(delta_E / temp): # random.random() 은 0과 1 사이 실수 반환. 나쁜 해로 이동할 확률 무작위 선택. 
                # 결국 T가 낮아질수록 e^E/T 는 확률이 낮아짐 0에 가까워짐 = 무작위로 나쁜해 받아들일 확률 낮아져감
                current_solution = next_solution
                current_value = next_value

        t += 1  # 반복 횟수 증가

    # 최종 해와 그 가치를 반환
    return current_solution, current_value

In [36]:
initial_state = Knapsack(items, capacity)
solution, total_value = simulated_annealing(initial_state)

# 결과 출력
selected_items = [items[i][0] for i in range(len(items)) if solution[i] == 1]
print(f"결과값: {total_value}, 아이템 목록: {solution}(={selected_items})")

초기값: 140
1번째 이웃상태: [110, 160, 150, 90, 80, 155, 240], 현재값: 140
2번째 이웃상태: [125, 175, 165, 105, 95, 140, 255], 현재값: 155
3번째 이웃상태: [65, 115, 105, 45, 155, 80, 195], 현재값: 95
4번째 이웃상태: [75, 125, 95, 55, 165, 90], 현재값: 105
5번째 이웃상태: [135, 185, 155, 115, 105, 150], 현재값: 165
6번째 이웃상태: [85, 135, 105, 165, 55, 100, 215], 현재값: 115
7번째 이웃상태: [75, 405, 125, 115, 155, 45, 90, 205], 현재값: 105
8번째 이웃상태: [15, 345, 65, 55, 95, 105, 30, 145], 현재값: 45
9번째 이웃상태: [65, 115, 105, 45, 155, 80, 195], 현재값: 95
10번째 이웃상태: [15, 345, 65, 55, 95, 105, 30, 145], 현재값: 45
11번째 이웃상태: [115, 165, 155, 195, 205, 130, 45], 현재값: 145
12번째 이웃상태: [135, 145, 225, 150, 65], 현재값: 165
13번째 이웃상태: [195, 205, 165, 210, 125], 현재값: 225
14번째 이웃상태: [225, 175, 205, 245, 135, 180, 95], 현재값: 195
15번째 이웃상태: [210, 160, 190, 230, 120, 195, 80], 현재값: 180
16번째 이웃상태: [170, 180, 130, 205, 90], 현재값: 190
17번째 이웃상태: [200, 190, 160, 220, 110, 185, 70], 현재값: 170
18번째 이웃상태: [190, 180, 170, 210, 100, 175, 60], 현재값: 160
19번째 이웃상태: [130, 400, 120, 110, 150, 1